In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append("/mnt/Data/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import os
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import colormaps as mycmps
import signale.tools as tools
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import scipy.signal as scsig
from scipy.stats import sem
from matplotlib import rcParams
import trajectory

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Wavelet Analysis Class

In [4]:
"""
A module which implements the continuous wavelet transform

---------------------------------------------------------
Code released under the BSD 3-clause licence.

Copyright (c) 2012, R W Fearick, University of Cape Town
All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
following conditions are met:

    * Redistributions of source code must retain the above copyright notice, this list of conditions and
        the following disclaimer.
    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and
        the following disclaimer in the documentation and/or other materials provided with the distribution.
    * Neither the name of the University of Cape Town nor the names of its contributors may be used to endorse
        or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR 
SERVICES;LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
WHETHER IN CONTRACT,STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE
USE OF THIS SOFTWARE,EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
---------------------------------------------------------

Wavelet classes:
Morlet
MorletReal
MexicanHat
Paul2      : Paul order 2
Paul4      : Paul order 4
DOG1       : 1st Derivative Of Gaussian
DOG4       : 4th Derivative Of Gaussian
Haar       : Unnormalised version of continuous Haar transform
HaarW      : Normalised Haar

Usage e.g.
wavelet=Morlet(data, largestscale=2, notes=0, order=2, scaling="log")
 data:  Numeric array of data (float), with length ndata.
        Optimum length is a power of 2 (for FFT)
        Worst-case length is a prime
 largestscale:
        largest scale as inverse fraction of length
        scale = len(data)/largestscalax1.set_xticklabels([e
        smallest scale should be >= 2 for meaningful data
 notes: number of scale intervals per octave
        if notes == 0, scales are on a linear increment
 order: order of wavelet for wavelets with variable order
        [Paul, DOG, ..]
 scaling: "linear" or "log" scaling of the wavelet scale.
        Note that feature width in the scale direction
        is constant on a log scale.
        
Attributes of instance:
wavelet.cwt:       2-d array of Wavelet coefficients, (nscales,ndata)
wavelet.nscale:    Number of scale intervals
wavelet.scales:    Array of scale values
                   Note that meaning of the scale will depend on the family
wavelet.fourierwl: Factor to multiply scale by to get scale
                   of equivalent FFT
                   Using this factor, different wavelet families will
                   have comparable scales

References:
A practical guide to wavelet analysis
C Torrance and GP Compo
Bull Amer Meteor Soc Vol 79 No 1 61-78 (1998)
naming below vaguely follows this.

updates:
(24/2/07):  Fix Morlet so can get MorletReal by cutting out H
(10/04/08): Numeric -> numpy
(25/07/08): log and lin scale increment in same direction!
            swap indices in 2-d coeffiecient matrix
            explicit scaling of scale axis
"""

class Cwt:
    """
    Base class for continuous wavelet transforms
    Implements cwt via the Fourier transform
    Used by subclass which provides the method wf(self,s_omega)
    wf is the Fourier transform of the wavelet function.
    Returns an instance.
    """

    fourierwl=1.00

    def _log2(self, x):
        # utility function to return (integer) log2
        return int( np.log(float(x))/ np.log(2.0)+0.0001 )

    def __init__(self, data, largestscale=1, notes=0, order=2, scaling='linear'):
        """
        Continuous wavelet transform of data

        data:    data in array to transform, length must be power of 2
        notes:   number of scale intervals per octave
        largestscale: largest scale as inverse fraction of length
                 of data array
                 scale = len(data)/largestscale
                 smallest scale should be >= 2 for meaningful data
        order:   Order of wavelet basis function for some families
        scaling: Linear or log
        """
        ndata = len(data)
        self.order=order
        self.scale=largestscale
        self._setscales(ndata,largestscale,notes,scaling)
        self.cwt= np.zeros((self.nscale,ndata), np.complex64)
        omega= np.array(range(0,ndata/2)+range(-ndata/2,0))*(2.0*np.pi/ndata)
        datahat=np.fft.fft(data)
        self.fftdata=datahat
        #self.psihat0=self.wf(omega*self.scales[3*self.nscale/4])
        # loop over scales and compute wvelet coeffiecients at each scale
        # using the fft to do the convolution
        for scaleindex in range(self.nscale):
            currentscale=self.scales[scaleindex]
            self.currentscale=currentscale  # for internal use
            s_omega = omega*currentscale
            psihat=self.wf(s_omega)
            psihat = psihat *  np.sqrt(2.0*np.pi*currentscale)
            convhat = psihat * datahat
            W    = np.fft.ifft(convhat)
            self.cwt[scaleindex,0:ndata] = W 
        return
    
    def _setscales(self,ndata,largestscale,notes,scaling):
        """
        if notes non-zero, returns a log scale based on notes per ocave
        else a linear scale
        (25/07/08): fix notes!=0 case so smallest scale at [0]
        """
        if scaling=="log":
            if notes<=0: notes=1 
            # adjust nscale so smallest scale is 2 
            noctave=self._log2( ndata/largestscale/2 )
            self.nscale=notes*noctave
            self.scales=np.zeros(self.nscale,float)
            for j in range(self.nscale):
                self.scales[j] = ndata/(self.scale*(2.0**(float(self.nscale-1-j)/notes)))
        elif scaling=="linear":
            nmax=ndata/largestscale/2
            self.scales=np.arange(float(2),float(nmax))
            self.nscale=len(self.scales)
        else: raise ValueError, "scaling must be linear or log"
        return
    
    def getdata(self):
        """
        returns wavelet coefficient array
        """
        return self.cwt
    def getcoefficients(self):
        return self.cwt
    def getpower(self):
        """
        returns square of wavelet coefficient array
        """
        return (self.cwt* np.conjugate(self.cwt)).real
    def getscales(self):
        """
        returns array containing scales used in transform
        """
        return self.scales
    def getnscale(self):
        """
        return number of scales
        """
        return self.nscale

# wavelet classes    
class Morlet(Cwt):
    """
    Morlet wavelet
    """
    _omega0=6.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0))*H
        return xhat

class MorletReal(Cwt):
    """
    Real Morlet wavelet
    """
    _omega0=5.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0)
                         + np.exp(-(s_omega+self._omega0)**2/2.0)
                         - np.exp(-(self._omega0)**2/2.0)
                         + np.exp(-(self._omega0)**2/2.0))
        return xhat
    
class Paul4(Cwt):
    """
    Paul m=4 wavelet
    """
    fourierwl=4* np.pi/(2.*4+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=0.11268723*s_omega[0:n/2]**4* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul2(Cwt):
    """
    Paul m=2 wavelet
    """
    fourierwl=4* np.pi/(2.*2+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=1.1547005*s_omega[0:n/2]**2* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul(Cwt):
    """
    Paul order m wavelet
    """
    def wf(self, s_omega):
        Cwt.fourierwl=4* np.pi/(2.*self.order+1.)
        m=self.order
        n=len(s_omega)
        normfactor=float(m)
        for i in range(1,2*m):
            normfactor=normfactor*i
        normfactor=2.0**m/ np.sqrt(normfactor)
        xhat= np.zeros(n)
        xhat[0:n/2]=normfactor*s_omega[0:n/2]**m* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class MexicanHat(Cwt):
    """
    2nd Derivative Gaussian (mexican hat) wavelet
    """
    fourierwl=2.0* np.pi/ np.sqrt(2.5)
    def wf(self, s_omega):
        # should this number be 1/sqrt(3/4) (no pi)?
        #s_omega = s_omega/self.fourierwl
        #print max(s_omega)
        a=s_omega**2
        b=s_omega**2/2
        return a* np.exp(-b)/1.1529702
        #return s_omega**2*exp(-s_omega**2/2.0)/1.1529702

class DOG4(Cwt):
    """
    4th Derivative Gaussian wavelet
    see also T&C errata for - sign
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(4.5)
    def wf(self, s_omega):
        return s_omega**4* np.exp(-s_omega**2/2.0)/3.4105319

class DOG1(Cwt):
    """
    1st Derivative Gaussian wavelet
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(1.5)
    def wf(self, s_omega):
        dog1= np.zeros(len(s_omega),complex64)
        dog1.imag=s_omega* np.exp(-s_omega**2/2.0)/sqrt(pi)
        return dog1

class DOG(Cwt):
    """
    Derivative Gaussian wavelet of order m
    but reconstruction seems to work best with +!
    """
    def wf(self, s_omega):
        try:
            from scipy.special import gamma
        except ImportError:
            print "Requires scipy gamma function"
            raise ImportError
        Cwt.fourierwl=2* np.pi/ np.sqrt(self.order+0.5)
        m=self.order
        dog=1.0J**m*s_omega**m* np.exp(-s_omega**2/2)/ np.sqrt(gamma(self.order+0.5))
        return dog

class Haar(Cwt):
    """
    Continuous version of Haar wavelet
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # 2/8/05 constants adjusted to match artem eim

    fourierwl=1.0#1.83129  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/4)**2/om
        return haar

class HaarW(Cwt):
    """
    Continuous version of Haar wavelet (norm)
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # normalised to unit power

    fourierwl=1.83129*1.2  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]#/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/2)**2/om
        return haar




In [5]:
animalPath = '/mnt/Data/data/Gerbils/'
localPath = '/mnt/Data/'
lfpPaths = []
for lfp in tools.locate('*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
lfpPaths

['/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-14_sleep1_csc4.lfp',
 '/mnt/D

### Colors & Basics

In [6]:
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
seabC = sns.xkcd_palette(colors)
seabP = sns.color_palette("Paired")
#sns.palplot(seabP)
#sns.palplot(sns.xkcd_palette(colors))
sns.set_style("ticks")
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'

## Basic lyout

In [7]:
figure = pl.figure(figsize = [15,10])
gs1 = gridspec.GridSpec(3, 4)
gs1.update(left=0.06, right=0.475,bottom=0.73,top=0.95,  wspace=0.05,hspace=0.01)
bx2 = pl.subplot(gs1[1:,0])
bx2up = pl.subplot(gs1[0,0])
bx3 = pl.subplot(gs1[1:,1])
bx3up = pl.subplot(gs1[0,1])
bx4 = pl.subplot(gs1[1:,2])
bx4up = pl.subplot(gs1[0,2])
bx5 = pl.subplot(gs1[1:,3])
bx5up = pl.subplot(gs1[0,3])
####################################
gs2 = gridspec.GridSpec(3, 4)
gs2.update(left=0.06, right=0.475,bottom=0.51,top=0.72,  wspace=0.05,hspace=0.01)
cx2 = pl.subplot(gs2[1:,0])
cx2up = pl.subplot(gs2[0,0])
cx3 = pl.subplot(gs2[1:,1])
cx3up = pl.subplot(gs2[0,1])
cx4 = pl.subplot(gs2[1:,2])
cx4up = pl.subplot(gs2[0,2])
cx5 = pl.subplot(gs2[1:,3])
cx5up = pl.subplot(gs2[0,3])
##################################
gs3 = gridspec.GridSpec(3, 4)
gs3.update(left=0.06, right=0.475,bottom=0.23,top=0.45, wspace=0.05,hspace=0.01)
ax2 = pl.subplot(gs3[1:,0])
ax2up = pl.subplot(gs3[0,0])
ax3 = pl.subplot(gs3[1:,1])
ax3up = pl.subplot(gs3[0,1])
ax4 = pl.subplot(gs3[1:,2])
ax4up = pl.subplot(gs3[0,2])
ax5 = pl.subplot(gs3[1:,3])
ax5up = pl.subplot(gs3[0,3])
###############################
gs3 = gridspec.GridSpec(3, 4)
gs3.update(left=0.06, right=0.475,bottom=0.01,top=0.22, wspace=0.05,hspace=0.01)
dx2 = pl.subplot(gs3[1:,0])
dx2up = pl.subplot(gs3[0,0])
dx3 = pl.subplot(gs3[1:,1])
dx3up = pl.subplot(gs3[0,1])
dx4 = pl.subplot(gs3[1:,2])
dx4up = pl.subplot(gs3[0,2])
dx5 = pl.subplot(gs3[1:,3])
dx5up = pl.subplot(gs3[0,3])

#axarr = [ax2,ax3,ax4,ax5]
#gs1.tight_layout(figure,rect=[0, 0.5, 0.5, 1])
#for axxx in axarr:
#    axxx.xaxis.tick_bottom()
#    axxx.yaxis.tick_left()
#gs3.tight_layout(figure,rect=[0.0, 0, 0.5, 0.33])
#bx4up.plot(range(10))

## Sleep phases

In [7]:
SleepEpochs = pd.read_pickle(animalPath+'SleepEpochs.pd')
df = pd.read_pickle(animalPath+'psdData.pd')

In [8]:
remEP = SleepEpochs[SleepEpochs.epoch=='REM']
swsEP = SleepEpochs[SleepEpochs.epoch=='SWS']
print (remEP.t1-remEP.t0).mean(), (remEP.t1-remEP.t0).min(), (remEP.t1-remEP.t0).max()
print (swsEP.t1-swsEP.t0).mean(),(swsEP.t1-swsEP.t0).min(),(swsEP.t1-swsEP.t0).max()

113.170212766 25 405
107.833333333 20 400


## Ripple Examples

In [9]:
rippDB = pd.read_pickle(animalPath+'RipplesTimeDB.pd')
c = np.int16(np.loadtxt('/mnt/Data/data/Gerbils/pink.clu.1'))[1:]
wc = np.int16(np.loadtxt('/mnt/Data/data/Gerbils/white.clu.1'))[1:]
np.place(wc,wc==3,[1])
np.place(c,c==2,[1])
np.place(c,c==3,[2])
rippDB['wc'] = wc
rippDB['c'] = c

In [14]:
print selS[selS.epoch=='rem'].shape,selG[selG.epoch=='rem'].shape
print np.unique(rippDB.date)
print len(lfpPaths)
print rippDB.shape
print selS.shape,selG.shape

(10, 24) (6, 24)
['2011-08-05' '2011-08-07' '2011-08-09' '2011-08-10' '2011-09-19'
 '2011-09-23']
26
(14481, 24)
(193, 24) (40, 24)


In [12]:
swrEx = np.logical_and(np.logical_and(rippDB.HilbertAbsPeak>5.9,rippDB.c==1),rippDB.wc==1).as_matrix()
gbsEx = np.logical_and(np.logical_and(rippDB.HilbertAbsPeak>11.9,rippDB.c==2),rippDB.wc==2).as_matrix()
selS = rippDB.loc[swrEx]
selG = rippDB.loc[gbsEx]

In [13]:
selG.shape,selS.shape

((40, 24), (193, 24))

In [56]:
fiz = pl.figure(figsize=[4,6])
bx = pl.Axes(fiz, [0., 0., 1., 1.])
bx.set_axis_off()
fiz.add_axes(bx)
cxx = bx.twinx()
pl.axis('off')
wavelet=Morlet
maxscale=3.05
notes=64
scaling="log" #or "linear"
#scaling="linear"
plotpower2d=True
# set up some data
Ns=2000
for kk,item in enumerate(selS[selS.epoch=='sws'].index):
    if kk>0:
        for path in lfpPaths:
            session = selS.session[item].split('0')[0]+selS.session[item].split('0')[1]
            if path.find(selS.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selS['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(5,2),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = 'magma',
                              #norm=mpl.colors.LogNorm(vmin= pwr.min()*20, vmax=.051*pwr.max()))
                              norm=mpl.colors.PowerNorm(gamma=0.7))
                #bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                #bx.axhline(100,linewidth='1.5',color = 'w',linestyle='--')
                #bx.axhline(50,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #bx.set_axis_off()
                cxx.set_aspect('auto')
                #ax4up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=0.9,alpha=0.000085)
                #cxx.plot(np.linspace(-0.512,0.512,1024),sig,c='w',lw=1.0)
                cxx.set_xlim(-.3,0.3)
                cxx.set_ylim(1.5*sig.min(),2*sig.max())
                pl.axis('off')
                fiz.savefig('/mnt/Data/data/Gerbils/swrSample-'+str(item)+'.png',transparent=True,dpi=200)
                cxx.cla()
                bx.cla()

In [15]:
bx2.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-1026.png'),aspect='auto',cmap = 'magma')
bx3.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-1119.png'),aspect='auto',cmap = 'magma')
bx4.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-1183.png'),aspect='auto',cmap = 'magma')
bx5.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-1406.png'),aspect='auto',cmap = 'magma')
cx2.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-2874.png'),aspect='auto',cmap = 'magma')
cx3.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-2887.png'),aspect='auto',cmap = 'magma')
cx4.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-3025.png'),aspect='auto',cmap = 'magma')
cx5.imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-3177.png'),aspect='auto',cmap = 'magma')
###################################3
ax2.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-3557.png'),aspect='auto',cmap = 'magma')
ax3.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-3579.png'),aspect='auto',cmap = 'magma')
ax4.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-3786.png'),aspect='auto',cmap = 'magma')
ax5.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-4216.png'),aspect='auto',cmap = 'magma')
dx2.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-4281.png'),aspect='auto',cmap = 'magma')
dx3.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-4418.png'),aspect='auto',cmap = 'magma')
dx4.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-4592.png'),aspect='auto',cmap = 'magma')
dx5.imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-4258.png'),aspect='auto',cmap = 'magma')

In [16]:
upArr = [bx2up,bx3up,bx4up,bx5up,cx2up,cx3up,cx4up,cx5up,ax2up,ax3up,ax4up,ax5up,dx2up,dx3up,dx4up,dx5up]
axarr = [bx2,bx3,bx4,bx5,cx2,cx3,cx4,cx5,ax2,ax3,ax4,ax5,dx2,dx3,dx4,dx5]

In [17]:
len(axarr)

16

In [18]:
swrIndex = []
for pngFile in tools.locate('*.png','/mnt/Data/data/Gerbils/swrEx/sel/'):
    swrIndex.append(int(pngFile[1].split('-')[-1].split('.')[0]))
fgbIndex = []
for pngFile in tools.locate('*.png','/mnt/Data/data/Gerbils/fgbEx/sel/'):
    fgbIndex.append(int(pngFile[1].split('-')[-1].split('.')[0]))
for axID,Idx in enumerate(np.unique(swrIndex)):
    axarr[axID].imshow(pl.imread('/mnt/Data/data/Gerbils/swrEx/sel/swrSample-'+str(Idx)+'.png'),aspect='auto',cmap = 'magma')
for axID,Idx in enumerate(np.unique(fgbIndex)):
    print axID+8
    axarr[axID+8].imshow(pl.imread('/mnt/Data/data/Gerbils/fgbEx/sel/fgbSample-'+str(Idx)+'.png'),aspect='auto',cmap = 'magma')
for item in zip(upArr[:8],np.unique(swrIndex)):
    session = rippDB.session[item[1]].split('0')[0] + rippDB.session[item[1]].split('0')[1]
    print item[1],session
    for path in lfpPaths:
        if path.find(rippDB.date[item[1]])>0 and path.find(session)>0:
            lfp = pkl.load(open(path,'rb'))
            tP = rippDB['t_peak'][item[1]]
            t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
            t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
            sig = lfp.signal[t1:t2]
            item[0].plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=1.0,alpha=0.95)
            item[0].set_xlim(-0.3,0.3)
for item in zip(upArr[8:],np.unique(fgbIndex)):
    session = rippDB.session[item[1]].split('0')[0] + rippDB.session[item[1]].split('0')[1]
    print item[1],session
    for path in lfpPaths:
        if path.find(rippDB.date[item[1]])>0 and path.find(session)>0:
            lfp = pkl.load(open(path,'rb'))
            tP = rippDB['t_peak'][item[1]]
            t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
            t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
            sig = lfp.signal[t1:t2]
            item[0].plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=1.0,alpha=0.95)
            item[0].set_xlim(-0.3,0.3)

8
9
10
11
12
13
14
15
1026 sleep3
1119 sleep3
1183 sleep3
1406 sleep3
2874 sleep3
2887 sleep3
3025 sleep3
3177 sleep3
3557 sleep1
3579 sleep1
3786 sleep2
4216 sleep3
4258 sleep3
4281 sleep3
4418 sleep3
4592 sleep1


In [19]:
ax2.set_xticks([0,1200,2400])
ax2up.set_xticks([])
ax2up.set_yticks([])
#ax2up.set_axis_off()
ax2.set_xticklabels([r'$-0.3$',r'$0$',r'$+0.3$'],fontsize=14)
ax2.set_xlabel('Time (s)',fontsize=14)
ax2.set_ylabel('Frequency (Hz)',fontsize=14)
#ax2up.set_ylabel('Voltage (uV)',fontsize=14)
ax2.set_yticks([600,1450,2200,3000])#,fontsize=14)
ax2.set_yticklabels([r'200',r'150',r'100',r'50'],fontsize=14)

In [20]:
for axx in axarr:
    axx.set_axis_off()

In [21]:
for axx in upArr:
    axx.set_xticks([])
    axx.set_yticks([])

In [22]:
pl.plot(sig)

In [45]:
g6S = SleepEpochs[SleepEpochs.animal=='G6']
g5S = SleepEpochs[SleepEpochs.animal=='G5']

In [76]:
remSum = 0
swsSum = 0
for idx,row in g5S[g5S.date=='2011-08-10'].iterrows():
    dt = row.t1-row.t0
    if row.epoch=='REM':
        remSum+=dt
    elif row.epoch=='SWS':
        swsSum+=dt
print remSum/60.0
print swsSum/60.0

9.41666666667
16.25


In [68]:
1371.0/swsSum, 3073.0/swsSum

(1.6780905752753978, 3.7613219094247246)

In [69]:
rippSWS = rippDB[rippDB.epoch=='sws']

In [83]:
SWS6 = rippSWS[(rippSWS.date=='2011-08-05')]
print SWS6.shape
print SWS6[(SWS6.c==1)& (SWS6.wc==1)].shape[0]#/float(swsSum)
print SWS6[(SWS6.c==2)& (SWS6.wc==2)].shape[0]#/float(swsSum)

(757, 24)
307
345


In [39]:
SWS5 = rippSWS[rippSWS.animal=='G5']
print SWS5.shape
print SWS5[(SWS5.c==1)& (SWS5.wc==1)].shape[0]/66.6/60
print SWS5[(SWS5.c==2)& (SWS5.wc==2)].shape[0]/66.6/60

(3078, 24)
0.17017017017
0.538288288288


In [66]:
print np.unique(SWS5.date),np.unique(SWS6.session)

['2011-08-05' '2011-08-07' '2011-08-09' '2011-08-10'] ['sleep02' 'sleep03']


In [71]:
np.unique(SWS6.animal)

array(['G5'], dtype=object)

In [78]:
SWS6

,t_i,t_f,peak_value,t_peak,epoch,date,session,animal,HilbertAbsPeak,HilbertPhasePeak,...,PC3,PC4,PC5,wPC1,wPC2,wPC3,wPC4,wPC5,wc,c
4518,849240,849282,10.3042,849264,sws,2011-08-10,sleep01,G5,3.542764,84.106219,...,0.516264,-0.333926,-1.795644,2.764674,2.318604,0.243378,-0.255952,-1.755444,2,2
4519,851637,851750,12.0409,851671,sws,2011-08-10,sleep01,G5,4.423406,36.845536,...,-0.140586,0.593121,1.434728,0.969845,0.222245,0.704069,0.125981,-0.705093,2,2
4520,853161,853220,16.0454,853180,sws,2011-08-10,sleep01,G5,6.454074,39.681836,...,0.694322,1.156534,0.109536,3.846935,-0.050228,-1.044772,-0.179286,-1.742876,2,2
4521,853257,853312,12.5049,853277,sws,2011-08-10,sleep01,G5,4.658692,43.188274,...,-0.367131,2.080759,0.165600,0.637196,0.987175,-0.236417,1.634838,-2.592286,2,2
4522,856438,856490,13.7389,856470,sws,2011-08-10,sleep01,G5,5.284459,124.765402,...,0.901329,0.519679,-0.384554,1.564245,-0.304014,-1.932574,-1.631506,-1.710653,2,2
4523,856507,856548,10.0524,856525,sws,2011-08-10,sleep01,G5,3.415085,330.019448,...,1.691285,-1.045913,-0.376511,3.981473,2.175348,-2.872427,1.038068,3.375428,2,2
4524,857140,857192,11.4794,857176,sws,2011-08-10,sleep01,G5,4.138721,148.882991,...,2.021951,-0.733143,1.189392,5.092594,-0.411447,-3.597580,-0.607780,3.581248,2,2
4525,858237,858286,10.2188,858253,sws,2011-08-10,sleep01,G5,3.499485,118.893834,...,0.207716,2.421929,-1.108999,3.117573,2.171282,-1.403794,2.607795,-1.685122,2,2
4526,860522,860567,11.8112,860541,sws,2011-08-10,sleep01,G5,4.306943,122.917024,...,-1.110043,-0.728102,-2.528610,4.136177,-0.080916,2.107175,-0.141334,-0.523192,2,2
4527,869760,869799,9.8304,869779,sws,2011-08-10,sleep01,G5,3.302514,34.367857,...,0.702617,0.576300,1.771809,1.235759,-2.011291,-1.766684,0.195068,0.761260,2,2
